In [8]:
# Importamos las librerías

import re
import nltk
import pandas as pd
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from nltk.tokenize import RegexpTokenizer

import gensim
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

###Diccionarios en español para el análisis de polaridad
nltk.download('punkt')
# Tokenización
# Descargar el conjunto de stopwords en español si no lo tienes
nltk.download('stopwords')
# Análisis de polaridad
# Obtención de listado de stopwords del inglés
stop_words = list(stopwords.words('spanish'))


[nltk_data] Downloading package punkt to /home/milton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/milton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_coments = pd.read_csv("twitter_coments_lemus.csv", usecols=["css-901oao 4", "css-901oao 11", "css-901oao 14"])
df_coments=df_coments.fillna('')
df_coments["texto"] = df_coments["css-901oao 4"] + ' ' + df_coments["css-901oao 11"] + ' ' + df_coments["css-901oao 14"]
df_coments = df_coments.drop("css-901oao 4", axis=1)
df_coments = df_coments.drop("css-901oao 11", axis=1)
df_coments = df_coments.drop("css-901oao 14", axis=1)
df_coments

,texto
0,"Que gusto poder acompañarlos el día de hoy, en..."
1,"Muchas gracias, amigo. Sin tu apoyo esto no hu..."
2,76
3,se han realizado no menos de cuatro reportes s...
4,Te felicito amigo @JuanJoseFrangie
...,...
815,"No creo, mira lo que han hecho en Zapopan esos..."
816,Animo Guadalajara!! @hagala_ @Trafico_ZMG
817,"Oye de mi gran amigo el Doc lomeli, no vas a e..."
818,@MilyRocha19 @felixgastelum1


In [10]:
def limpiar_texto(texto):
    # Eliminar caracteres especiales y números
    texto_limpio = re.sub(r'[^\w\sáéíóúüÁÉÍÓÚÜñ]', '', texto, flags=re.UNICODE)
    # Eliminar espacios en blanco adicionales y convertir a minúsculas
    texto_limpio = ' '.join(texto_limpio.split()).lower()
    return texto_limpio

df_coments["texto"] = df_coments["texto"].apply(limpiar_texto)
df_coments_clean = df_coments.copy()
df_coments_clean

,texto
0,que gusto poder acompañarlos el día de hoy en ...
1,muchas gracias amigo sin tu apoyo esto no hubi...
2,76
3,se han realizado no menos de cuatro reportes s...
4,te felicito amigo juanjosefrangie
...,...
815,no creo mira lo que han hecho en zapopan esos ...
816,animo guadalajara hagala_ trafico_zmg
817,oye de mi gran amigo el doc lomeli no vas a es...
818,milyrocha19 felixgastelum1


In [13]:
df_coments['texto'] = df_coments['texto'].str.lower()
tokenizer = RegexpTokenizer(r'\w+')
df_coments['tokens'] = df_coments['texto'].apply(tokenizer.tokenize)
df_coments.to_csv("coments_x.csv")
df_coments

,texto,tokens
0,que gusto poder acompañarlos el día de hoy en ...,"[que, gusto, poder, acompañarlos, el, día, de,..."
1,muchas gracias amigo sin tu apoyo esto no hubi...,"[muchas, gracias, amigo, sin, tu, apoyo, esto,..."
2,76,[76]
3,se han realizado no menos de cuatro reportes s...,"[se, han, realizado, no, menos, de, cuatro, re..."
4,te felicito amigo juanjosefrangie,"[te, felicito, amigo, juanjosefrangie]"
...,...,...
815,no creo mira lo que han hecho en zapopan esos ...,"[no, creo, mira, lo, que, han, hecho, en, zapo..."
816,animo guadalajara hagala_ trafico_zmg,"[animo, guadalajara, hagala_, trafico_zmg]"
817,oye de mi gran amigo el doc lomeli no vas a es...,"[oye, de, mi, gran, amigo, el, doc, lomeli, no..."
818,milyrocha19 felixgastelum1,"[milyrocha19, felixgastelum1]"
